# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



How to stream json using a {match} function?

In [ ]:
// initialize
var path = require('path');
var fs = require('fs');

// provide a function for importing any notebook as a module and executing it
var getCellsOrDirectory = (filename) => {
    const fname = filename.replace('.ipynb', '');
    if (fs.existsSync(fname)
        && fs.statSync(fname).isDirectory()) {
        const files = fs.readdirSync(fname)
            .sort((a, b) => parseInt(a.replace(/[^0-9]/ig, '')) - parseInt(b.replace(/[^0-9]/ig, ''))) //sort numerically
            .filter(f => f.indexOf('.js') > -1);
        return files.map(f => fs.readFileSync(path.join(fname, f)).toString());
    }
    else {
        return getCells(filename, ['javascript', 'code'])
            .map((c, i) => ({
                fresher: c.source.join(''),
                id: path.basename(filename) + '[' + i + ']',
                filename: filename
            }));
    }
};

var executeCell = (c, ctx, i) => {
    console.log(i);
    const obj = {};
    const key = c.id;
    return promiseNewContext(c.fresher, ctx, key)
        .then(r => {
            if (typeof r === 'function') {
                obj[key] = r;
                obj[r.name] = obj[key];
            }
            obj[i] = r;
            return obj;
        })
    .catch(e => console.log(e))
}

// output nothing here, cached version of this function is assigned below.
if (typeof imported !== 'object' || imported === null) {
    var imported = {};
}

// How to test if a notebook has already been imported?
var importNotebook = (notebook, ctx = {}) => {
    console.log(notebook);
    if (typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    if (typeof imported[notebook] !== 'undefined') {
        return imported[notebook];
    }
    console.log('hit');
    var cells;
    if(fs.existsSync(path.resolve(__dirname + '/' + notebook))) {
        const cells = getCellsOrDirectory(path.resolve(__dirname + '/' + notebook));
        return runAllPromises(cells
            .map((c, i) => resolve => executeCell(c, ctx, i)
                 .then(r => resolve(r))
                 .catch(e => {console.log(e); resolve()})))
        .then(results => {
        console.log(results.length);
            if(typeof results.reduce !== 'undefined') {
                return results.reduce((obj, r) => Object.assign(obj, r), {});
            } else {
                return results[Object.keys(results).pop()];
            }
        })
        .catch(e => console.log(e))
    }
    return (typeof cells !== 'undefined' ? Promise.resolve([]) : interpret(notebook))
        .then(cells => {
        console.log(cells.length);
            if( typeof cells.fresher !== 'undefined') {
                return executeCell(cells, ctx, cells.id);
            } else {
                return runAllPromises(cells
                    .map((c, i) => resolve => executeCell(c, ctx, i)
                         .then(r => resolve(r))
                         .catch(e => console.log(e))));
            }
        })
        .then(results => {
            if(typeof results.reduce !== 'undefined') {
                return results.reduce((obj, r) => Object.assign(obj, r), {});
            } else {
                return results[Object.keys(results).pop()];
            }
        })
        .catch(e => console.log(e))
};

// $$.done(); for our dumb parser



In [ ]:
// initialize
var fs = require('fs');var JSONStream = require('JSONStream');var jp = require('jsonpath');

/*var streamJson = (file, parse, match = ((c) => true)) => {
    var stream = fs.createReadStream(file)        .pipe(JSONStream.parse(parse));
    stream.on('data', (m) => match(m));
    console.log(file);
    // create a promise out of this stream    return new Promise((resolve, reject) => {        stream.on('error', e => reject(e));        stream.on('close', () => resolve());    });};
*/

var streamJson = (file, path, match = ((c) => true)) => {
    
    const json = JSON.parse(fs.readFileSync(file).toString());
    const nodes = jp.nodes(json, '$..*');
    return nodes.filter(n => typeof n.value === 'object' && n.value !== null).forEach(n => match({
        path: n.path.slice(1),
        value: n.value}));
};
// $$.done() for our dumb parser


How to parse cells of {types} from a notebook?



In [ ]:
// initialize// How to use getCells?var getCells = (notebook, types = ['*', 'code']) => {
    var kernel;
    var cells = [];
    streamJson(notebook, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'metadata'
            && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if (match.path[0] === 'cells'
            && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    });
    if (types.indexOf(kernel.language) === -1
        && types.indexOf('*') === -1) {
        console.log('tried to import ' + types + JSON.stringify(kernel));
        return [];
    }
    return cells.map(c => Object.assign(c, {language: kernel.language}));
};// $$.done() for our dumb parser

How to run all promises sequentially?



In [ ]:
// initializevar runAllPromises = (promises) => {    return promises.reduce((promise, func) => {        return promise.then(result => {            return (typeof func == 'function'                ? (new Promise(func)) : Promise.resolve(func))                .then(Array.prototype.concat.bind(result));        });    }, Promise.resolve([]));};// $$.done() for our dumb parser

How to load interpreter?

How to search notebooks?


In [ ]:
// initialize
var fs = require('fs');

// save for later use
var fuseSearch, interpretObject;
var runInNewContext = (code, ctx) => void 0;

var interpret = (s) => {
    var searches = typeof s === 'string' ? [s] : s;
    return importNotebook('interpret all notebooks.ipynb')
        .then((f) => {
            fuseSearch = f['fuseSearch'];
            interpretObject = f['interpretObject'];
        })
        .then(() => runAllPromises(searches.map(q => resolve => {
            const query = (/([^\[]*)\[(.*?)]/ig).exec(q);
            if (query) {
                // TODO: filter query results by filename
                return fuseSearch(query[2])
                    .then(r => {
                        return r.filter(id =>
                            id.substr(0, query[1].length + 1) === query[1] + '[')
                    })
                    .then(r => resolve(typeof s === 'string' ? r : [r]))
            } else {
                return fuseSearch(q)
                    .then(r => resolve(typeof s === 'string' ? r : [r]))
            }
        })))
        .then(r => {
            if (r.length === 0) {
                throw 'Nothing found for ' + s;
            }
            if (typeof s === 'string' && r[0].substr(0, s.length + 1) === s + '[') {
                return interpretObject([r]);
            }
            return typeof s === 'string'
                ? interpretObject([r[0]])
                : interpretObject(r.map(r => r[0]))
        })
        .then(r => {
            return r.map(r => {
                if (typeof r === 'undefined') {
                    return r;
                }
                // must have a unique id for each unique cell so that
                // individual cells can serve as modules as well as notebooks
                // adding the cell id as a part of the filename
                r.runInNewContext = (ctx) => runInNewContext(
                    r.fresher,
                    Object.assign(ctx || {}, {
                        useCache: false,
                        __filename: r.filename + r.id
                    }),
                    {filename: r.filename + r.id}, false);
                return r;
            })
        })
        .then(r => typeof s === 'string' ? r[0] : r)
        .catch(e => console.log(e));
};

// $$.done() for our dumb parser


Import arbitrary code in to the current context?

In [ ]:
// initialize
var Module = require('module').Module;var path = require('path');var requireCode = (code, pathToCode, tmpGlobal) => {    // must have a new name for every generation otherwise cache will be returned    var filepath = path.resolve(process.cwd(), pathToCode);    var dirname = path.dirname(filepath);    var cachedModule = Module._cache[filepath];    if (cachedModule && tmpGlobal.useCache !== false) {        tmpGlobal.module = cachedModule;        return cachedModule.exports;    }    var mod = new Module(filepath, global.module);    tmpGlobal.module = mod;    Module._cache[filepath] = mod;    mod.filename = filepath;
    mod.paths = Module._nodeModulePaths(dirname);    mod._compile(code, filepath);    mod.loaded = true;    tmpGlobal.exports = mod.exports;    return mod.exports;};// $$.done() for our dumb parser

Import arbitrary code in to a new context?

In [ ]:
// initialize
var Module = require('module').Module;
var path = require('path');var vm = require('vm');var runner = function (resolve, reject, file) {    'use strict';    this.resolve = resolve;    this.reject = reject;    this.isAsync = false;    this.returnVal = null;    this.file = file;    this.async = () => {        this.isAsync = true;    };    this.sendResult = (r) => {        if (this.isAsync) {            return this.resolve(r);        } else {            this.returnVal = r;        }    };    this.sendError = (e) => {        console.log(e);        this.reject(e);    }};var promiseNewContext = (c, ctx, newFilename) => {
    return new Promise((resolve, reject) => {        var isPromise = false;        var run = new runner(resolve, reject, newFilename);        var newCtx = Object.assign(ctx || {}, {
            useCache: false,            __filename: newFilename,            $$: run        });
        const result = runInNewContext(c, newCtx, {
            filename: newFilename
        }, false);        if (typeof result === 'object' && result !== null) {            if (result.constructor === Promise.prototype.constructor) {                isPromise = true; // will return result of promise            }        }        return Promise.resolve(result)            .then(r => {                if (!run.isAsync) {                    return resolve(run.returnVal || r);                } else if (isPromise) {                    return resolve(run.returnVal || r);                }                return r;            })            .catch(e => reject(e))    })};var runInNewContext = (code, ctx, options, isModule = true) => {    var tmpGlobal = ctx || {};    tmpGlobal.global = tmpGlobal;    Object.assign(tmpGlobal, ctx);
    Object.assign(tmpGlobal, {        'console': console,        'process': process,        'Promise': Promise,        'Buffer': Buffer,        'clearImmediate': clearImmediate,        'clearInterval': clearInterval,        'clearTimeout': clearTimeout,        'setImmediate': setImmediate,        'setInterval': setInterval,        'setTimeout': setTimeout,        '__filename': path.basename(ctx.__filename),        '__dirname': path.dirname(ctx.__filename),        'require': require    });    Object.assign(options || {}, {lineOffset: -1});    if (!isModule) {        tmpGlobal.module = {};    }
    if(isModule) {
        var result = requireCode('module.exports=(function(ctx) { Object.assign(global, ctx); \r' + code + '\r})', path.join(ctx.__dirname, ctx.__filename), tmpGlobal)(tmpGlobal);
        return tmpGlobal.global.module.exports || result;
    } else {
        var result = vm.runInNewContext(code, tmpGlobal, options);
        module.paths = Module._nodeModulePaths(path.join(ctx.__dirname, ctx.__filename));
        return tmpGlobal.global.module.exports || result;
    }};// $$.done() for our dumb parser

In [ ]:
// initializeif (typeof imported !== 'object' || imported === null) {    var imported = {};}imported['import notebook.ipynb'] = {    'import notebook.ipynb[2]': streamJson,    'streamJson': streamJson,    'import notebook.ipynb[3]': getCells,    'getCells': getCells,    'import notebook.ipynb[4]': runAllPromises,    'runAllPromises': runAllPromises,    'import notebook.ipynb[7]': importNotebook,    'import': importNotebook,    'import notebook.ipynb[8]': imported,    'imported': imported,    'import notebook.ipynb[9]': interpret,    'interpret': interpret,    'import notebook.ipynb[11]': runInNewContext,    'runInNewContext': runInNewContext};module.exports = imported['import notebook.ipynb'];// $$.done() for our dumb parser

These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

